In [51]:
LANGSMITH_TRACING=True
LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
LANGSMITH_API_KEY="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"
LANGSMITH_PROJECT="pr-mundane-mining-97"


In [52]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [53]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_ollama import ChatOllama


llm=ChatOllama(model="llama3.1")

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

tools

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x10dfc3ef0>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x10dfc3ef0>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x10dfc3ef0>),
 QuerySQLCheckerTool(description='Use this tool to double check if your 

In [54]:
response = llm.invoke('Hello')
print(response)

content='How are you today? Is there something I can help you with or would you like to chat?' additional_kwargs={} response_metadata={'model': 'llama3.1', 'created_at': '2025-02-16T06:55:52.015578Z', 'done': True, 'done_reason': 'stop', 'total_duration': 9839302500, 'load_duration': 67117125, 'prompt_eval_count': 11, 'prompt_eval_duration': 5879000000, 'eval_count': 21, 'eval_duration': 3887000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-0c9d10fd-d428-4ba2-9974-ae3a40d88cb9-0' usage_metadata={'input_tokens': 11, 'output_tokens': 21, 'total_tokens': 32}


In [55]:
from langchain import hub

prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

assert len(prompt_template.messages) == 1
prompt_template.messages[0].pretty_print()

/Users/nizar/Documents/Projects/leads_automation/.venv/lib/python3.12/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


================================ System Message ================================

You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to th

In [56]:
system_message = prompt_template.format(dialect="SQLite", top_k=5)

In [ ]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

system_message = (
    "You are an agent designed to interact with a SQLite database with access to these tools [sql_db_query, sql_db_schema, sql_db_list_tables, sql_db_query_checker]."
    "First read the users input and think of a plan that consists of a series of seperate tool calls:"
    "1. First use tool named sql_db_list_tables to list all the table names in the database and identify the correct table syntax."
    "2. Select the most relevant and VALID tables from the output of sql_db_list_tables tool call"
    "and then use the tool sql_db_schema (Example Input: table1, table2, table3') to understand their DDL so you can select the relevant columns."
    "3. Based on all this analysis from step 1 and 2 create a syntactically correct SQLite query to help answer the user input"
    "4. Use the sql_db_query_checker to check it first before running the query to double check it's syntax"
    "5 After the checked tool say's its correct you may run the sql query with sql_db_query tool" 
    "6. then look at the results of the query and return the answer."
    "7. If the answer is wrong or inaccurate you should reflect on your previous work and improve it until a satisfactory answer has been reached."
)

agent_executor = create_react_agent(llm, tools, prompt=system_message)

In [63]:
question = "How many employees are there?"

for step in agent_executor.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

How many employees are there?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (9f65820f-5abc-4dd6-8030-5976732ff41b)
 Call ID: 9f65820f-5abc-4dd6-8030-5976732ff41b
  Args:
    tool_input: 
  sql_db_schema (64fad982-0202-40ba-8f20-ccdf669a81cb)
 Call ID: 64fad982-0202-40ba-8f20-ccdf669a81cb
  Args:
    table_names: employees
  sql_db_query_checker (4d15cc07-c7c4-460c-a141-b19b63663229)
 Call ID: 4d15cc07-c7c4-460c-a141-b19b63663229
  Args:
    query: SELECT COUNT(employee_id) FROM employees
  sql_db_query (49a0a265-8dde-4380-8639-36909507ac96)
 Call ID: 49a0a265-8dde-4380-8639-36909507ac96
  Args:
    query: SELECT COUNT(employee_id) FROM employees
================================= Tool Message =================================
Name: sql_db_query

Error: (sqlite3.OperationalError) no such table: employees
[SQL: SELECT COUNT(employee_id) FROM